In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data


from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu2/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [ ]:
path = "data/dogscatsreduxkernelsedition/"
model_path = path + 'models/'
# results_path
if not os.path.exists(model_path): os.mkdir(model_path)

In [ ]:
from shutil import copyfile
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [ ]:
% cd data/dogscatsreduxkernelsedition
% cd train


In [ ]:
g  = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i],'../valid/' + shuf[i])

In [ ]:
% mkdir ../sample
% mkdir ../sample/train
% mkdir ../sample/valid

In [ ]:
g  = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): copyfile(shuf[i],'../sample/train/' + shuf[i])

In [ ]:
% cd ../valid

In [ ]:
g  = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): copyfile(shuf[i],'../sample/valid/' + shuf[i])

In [ ]:
% cd train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

In [ ]:
% cd valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

In [ ]:
% cd sample/train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

In [ ]:
%cd sample/valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

In [ ]:
% cd test
% mkdir unknown
% mv *.jpg unknown/


In [ ]:
batch_size=100
#batch_size=4

from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [ ]:
# vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
vgg.model.save_weights(path+'models/ft1.h5')

batches, preds = vgg.test(path+'test', batch_size = batch_size)

In [ ]:
filenames = batches.filenames
save_array(path+'results/test_preds.dat',preds)
save_array(path+'results/filenames.dat',filenames)
preds = load_array('results/test_preds.dat')
filenames = load_array('results/filenames.dat')
isdog = preds[:,1]
ids =[int(f[8:f.find('.')]) for f in filenames]
subm = np.stack([ids,isdog], axis = 1)

np.savetxt('data/dogscatsreduxkernelsedition/subm98.csv' subm, fmt='%d,%.5f', header='id,label', comments='')  

from IPython.display import FileLink
FileLink('data/dogscatsreduxkernelsedition/subm98.csv')

# manual

In [ ]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

In [ ]:
val_data = get_data(val_batches)

In [ ]:
trn_data = getdata(baches)

In [ ]:
save_array(model_path+ 'train_data.bc', trn_data)
save_array(model_path + 'valid_data.bc', val_data)

In [ ]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [ ]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [ ]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
trn_features = model.predict(trn_data, batch_size = batch_size)
val_features = model.predict(val_data, batch_size = batch_size)

In [ ]:
save_array(model_path+ 'train_lastlayer_features.bc', trn_features)
save_array(model_path + 'valid_lastlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

In [ ]:
lm = Sequential([Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss = 'categorical_crossentropy', metrics=['accuracy'] )

In [ ]:
batch_size = 64

In [ ]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

In [ ]:
# We want both the classes...
preds = lm.predict_classes(val_features, batch_size=batch_size)
# ...and the probabilities of being a cat
probs = lm.predict_proba(val_features, batch_size=batch_size)[:,0]
probs[:8]

In [ ]:
model.pop()
for layer in model.layers: layer.trainable=False

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
??vgg.finetune

In [ ]:
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data,trn_labels,batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

In [ ]:
def fit_model(model, batches, val_batches, nb_eopch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch,
                       validation_data=val_batches, nb_val_samples = val_batches.N)

In [ ]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical)crossentropy', metrics=['accuracy'])

In [ ]:
fit_model(model, batches, val_batches, nb_epoch=2)

In [ ]:
model.save_weights(model_path+'finetune1.h5')

In [ ]:
model.load_weights(model_path+'finetune1.h5')

In [ ]:
model.evaluate(val_data, val_labels)

In [ ]:
preds = model.predict_classes(val_data, batch_size=batch_size)
probs = model.predict_proba(val_data, batch_size=batch_size)[:,0]
probs[:8]

In [ ]:
layers = model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable=True

In [ ]:
K.set_value(opt.lr, 0.01)
fit_model(model, batches, val_batches, 3)

In [ ]:
model.save_weights(model_path+'finetune2.h5')

In [ ]:
for layer in layers[12:]: layer.trainable=True
K.set_value(opt.lr, 0.001)

In [ ]:
fit_model(model, batches, val_batches, 4)

In [ ]:
model.save_weights(model_path+'finetune3.h5')

In [ ]:
model.load_weights(model_path+'finetune2.h5')
model.evaluate_generator(get_batches('valid', gen, False, batch_size*2), val_batches.N)